In [1]:
import math
import os
from collections import Counter
from functools import partial
from multiprocessing import Pool

import networkx as nx
import torch
import torch.nn as nn
import tqdm
from torch_geometric.data import Batch, Data

# from torch.utils.data import DataLoader, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_networkx

from Configures import mcts_args
from utils import PlotUtils

In [2]:
class MCTSNode:

    def __init__(
        self,
        coalition: list,
        data: Data,
        ori_graph: nx.Graph,
        c_puct: float = 10.0,
        W: float = 0,
        N: int = 0,
        P: float = 0,
    ):
        self.data = data
        self.coalition = coalition
        self.ori_graph = ori_graph
        self.c_puct = c_puct
        self.children = []
        self.W = W  # sum of node value
        self.N = N  # times of arrival
        self.P = P  # property score (reward)

    def Q(self):
        return self.W / self.N if self.N > 0 else 0

    def U(self, n):
        return self.c_puct * self.P * math.sqrt(n) / (1 + self.N)

In [3]:
def mcts_rollout(tree_node, state_map, data, graph, score_func):
    cur_graph_coalition = tree_node.coalition
    if len(cur_graph_coalition) <= mcts_args.min_atoms:
        return tree_node.P

    # Expand if this node has never been visited
    if len(tree_node.children) == 0:
        node_degree_list = list(graph.subgraph(cur_graph_coalition).degree)
        node_degree_list = sorted(
            node_degree_list, key=lambda x: x[1], reverse=mcts_args.high2low
        )
        all_nodes = [x[0] for x in node_degree_list]

        if len(all_nodes) < mcts_args.expand_atoms:
            expand_nodes = all_nodes
        else:
            expand_nodes = all_nodes[: mcts_args.expand_atoms]

        for each_node in expand_nodes:
            # for each node, pruning it and get the remaining sub-graph
            # here we check the resulting sub-graphs and only keep the largest one
            subgraph_coalition = [node for node in all_nodes if node != each_node]

            subgraphs = [
                graph.subgraph(c)
                for c in nx.connected_components(graph.subgraph(subgraph_coalition))
            ]
            main_sub = subgraphs[0]
            for sub in subgraphs:
                if sub.number_of_nodes() > main_sub.number_of_nodes():
                    main_sub = sub

            new_graph_coalition = sorted(list(main_sub.nodes()))

            # check the state map and merge the same sub-graph
            Find_same = False
            for old_graph_node in state_map.values():
                if Counter(old_graph_node.coalition) == Counter(new_graph_coalition):
                    new_node = old_graph_node
                    Find_same = True

            if Find_same == False:
                new_node = MCTSNode(new_graph_coalition, data=data, ori_graph=graph)
                state_map[str(new_graph_coalition)] = new_node

            Find_same_child = False
            for cur_child in tree_node.children:
                if Counter(cur_child.coalition) == Counter(new_graph_coalition):
                    Find_same_child = True

            if Find_same_child == False:
                tree_node.children.append(new_node)

        scores = compute_scores(score_func, tree_node.children)
        for child, score in zip(tree_node.children, scores):
            child.P = score

    sum_count = sum([c.N for c in tree_node.children])
    selected_node = max(tree_node.children, key=lambda x: x.Q() + x.U(sum_count))
    v = mcts_rollout(selected_node, state_map, data, graph, score_func)
    selected_node.W += v
    selected_node.N += 1
    return v


def mcts(data, gnnNet, prototype):
    data = Data(x=data.x, edge_index=data.edge_index)
    graph = to_networkx(data, to_undirected=True)
    data = Batch.from_data_list([data])
    num_nodes = graph.number_of_nodes()
    root_coalition = sorted([i for i in range(num_nodes)])
    root = MCTSNode(root_coalition, data=data, ori_graph=graph)
    state_map = {str(root.coalition): root}
    score_func = partial(gnn_prot_score, data=data, gnnNet=gnnNet, prototype=prototype)
    for rollout_id in range(mcts_args.rollout):
        mcts_rollout(root, state_map, data, graph, score_func)

    explanations = [node for _, node in state_map.items()]
    explanations = sorted(explanations, key=lambda x: x.P, reverse=True)
    explanations = sorted(explanations, key=lambda x: len(x.coalition))

    result_node = explanations[0]
    for result_idx in range(len(explanations)):
        x = explanations[result_idx]
        if len(x.coalition) <= mcts_args.max_atoms and x.P > result_node.P:
            result_node = x

    # compute the projected prototype to return
    mask = torch.zeros(data.num_nodes).type(torch.float32)
    mask[result_node.coalition] = 1.0
    ret_x = data.x * mask.unsqueeze(1)
    ret_edge_index = data.edge_index
    mask_data = Data(x=ret_x, edge_index=ret_edge_index)
    mask_data = Batch.from_data_list([mask_data])
    _, _, _, emb, _ = gnnNet(mask_data, protgnn_plus=False)
    return result_node.coalition, result_node.P, emb


def compute_scores(score_func, children):
    results = []
    for child in children:
        if child.P == 0:
            score = score_func(child.coalition)
        else:
            score = child.P
        results.append(score)
    return results


def gnn_prot_score(coalition, data, gnnNet, prototype):
    """the similarity value of subgraph with selected nodes"""
    epsilon = 1e-4
    mask = torch.zeros(data.num_nodes).type(torch.float32)
    mask[coalition] = 1.0
    ret_x = data.x * mask.unsqueeze(1)
    ret_edge_index = data.edge_index
    # row, col = data.edge_index
    # edge_mask = (mask[row] == 1) & (mask[col] == 1)
    # ret_edge_index = data.edge_index[:, edge_mask]

    mask_data = Data(x=ret_x, edge_index=ret_edge_index)
    mask_data = Batch.from_data_list([mask_data])
    _, _, _, emb, _ = gnnNet(mask_data, protgnn_plus=False)
    distance = torch.norm(emb - prototype) ** 2
    similarity = torch.log((distance + 1) / (distance + epsilon))
    return similarity.item()

In [4]:
import numpy as np

from Configures import data_args, model_args, train_args
from load_dataset import get_dataloader, get_dataset
from models import GnnNets, GnnNets_NC

print("start loading data====================")
dataset = get_dataset(data_args.dataset_dir, data_args.dataset_name)
input_dim = dataset.num_node_features
output_dim = int(dataset.num_classes)
dataloader = get_dataloader(
    dataset, train_args.batch_size, data_split_ratio=data_args.data_split_ratio
)
data_indices = dataloader["train"].dataset.indices
print("start training model==================")
gnnNets = GnnNets(input_dim, output_dim, model_args)
prototype_shape = (output_dim * model_args.num_prototypes_per_class, 128)
prototype_vectors = nn.Parameter(torch.rand(prototype_shape), requires_grad=False)
checkpoint = torch.load("./checkpoint/ba2motifs/gin_best.pth", weights_only=False)
gnnNets.update_state_dict(checkpoint["net"])
gnnNets
gnnNets.eval()

save_dir = os.path.join(
    "./new_results", f"{mcts_args.dataset_name}_" f"{model_args.model_name}_"
)
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
plotutils = PlotUtils(dataset_name=data_args.dataset_name)

batch_indices = np.random.choice(data_indices, 1, replace=False)

start loading data====================


start training model==================


Processing...
Done!


In [5]:
import matplotlib.pyplot as plt


def plot_graph(G, title=None):
    nx.draw(
        to_networkx(G, to_undirected=True),
        with_labels=False,
        node_color="#3b8bc2",
        node_size=500,
        edge_color="k",
        linewidths=2,
        font_size=15,
    )
    if title:
        plt.title(title)
    plt.show()

In [6]:
def plot_mutag(
    graph1,
    graph2=None,
    original_x1=None,
    perm1=None,
    original_x2=None,
    perm2=None,
    with_labels=False,
):
    import matplotlib.pyplot as plt
    import networkx as nx
    import matplotlib
    import matplotlib.patches as mpatches
    from torch_geometric.utils import to_networkx

    colormap = matplotlib.colormaps.get_cmap("Pastel1")

    color_map = {
        0: colormap(0),  # C
        1: colormap(1),  # O
        2: colormap(2),  # Cl
        3: colormap(3),  # H
        4: colormap(4),  # N
        5: colormap(5),  # F
        6: colormap(6),  # Br
        7: colormap(7),  # S
        8: colormap(8),  # P
        9: colormap(9),  # I
        "other": "gray",
    }

    atom_types = {
        0: "C",
        1: "O",
        2: "Cl",
        3: "H",
        4: "N",
        5: "F",
        6: "Br",
        7: "S",
        8: "P",
        9: "I",
    }

    def plot_single_graph(graph, ax, original_x=None, perm=None):
        G = to_networkx(graph, to_undirected=True)

        node_colors = []
        node_labels = {}

        if original_x is not None and perm is not None:
            mapped_x = original_x[perm[: graph.num_nodes]]
            for node in range(graph.num_nodes):
                one_hot = mapped_x[node].tolist()
                try:
                    node_type = one_hot.index(1)
                except ValueError:
                    node_type = "other"
                node_colors.append(color_map.get(node_type, "gray"))
                node_labels[node] = perm[node].item()
        else:
            for node in range(graph.num_nodes):
                one_hot = graph.x[node].tolist()
                try:
                    node_type = one_hot.index(1)
                except ValueError:
                    node_type = "other"
                node_colors.append(color_map.get(node_type, "gray"))
                node_labels[node] = perm[node].item() if perm is not None else node

        pos = nx.spring_layout(G)
        nx.draw(
            G,
            pos,
            node_color=node_colors,
            with_labels=with_labels,
            labels=node_labels,
            node_size=500,
            font_weight="bold",
            ax=ax,
        )

    legend_handles = [
        mpatches.Patch(color=colormap(0), label="C"),
        mpatches.Patch(color=colormap(1), label="O"),
        mpatches.Patch(color=colormap(2), label="Cl"),
        mpatches.Patch(color=colormap(3), label="H"),
        mpatches.Patch(color=colormap(4), label="N"),
        mpatches.Patch(color=colormap(5), label="F"),
        mpatches.Patch(color=colormap(6), label="Br"),
        mpatches.Patch(color=colormap(7), label="S"),
        mpatches.Patch(color=colormap(8), label="P"),
        mpatches.Patch(color=colormap(9), label="I"),
        mpatches.Patch(color="gray", label="Other"),
    ]

    if graph2 is None:
        fig, ax = plt.subplots(figsize=(8, 8))
        plot_single_graph(graph1, ax, original_x1, perm1)
    else:
        fig, axes = plt.subplots(1, 2, figsize=(24, 16))
        plot_single_graph(graph1, axes[0], original_x1, perm1)
        plot_single_graph(graph2, axes[1], original_x2, perm2)

    fig.legend(handles=legend_handles, loc="lower left", title="Node Types")
    plt.show()

In [8]:
def find_best_prototype(data, gnnNet, prototype_vectors, num_prototypes):
    """
    Find the best prototype for the given graph data.
    This runs MCTS for each prototype and returns the one with the highest similarity score.
    """
    best_prototype_index = -1
    best_coalition = None
    best_similarity = -float("inf")
    best_prototype_embedding = None

    for prototype_index in range(num_prototypes):
        coalition, similarity, prototype_embedding = mcts(
            data, gnnNet, prototype_vectors[prototype_index]
        )

        if similarity > best_similarity:
            best_similarity = similarity
            best_prototype_index = prototype_index
            best_coalition = coalition
            best_prototype_embedding = prototype_embedding

    return best_prototype_index, best_coalition, best_prototype_embedding


import matplotlib.pyplot as plt
import pickle
from torch_geometric.utils import to_networkx, from_networkx
from torch_geometric.utils import subgraph


def save_prototype_subgraph(data, gnnNet, prototype_vectors):
    # Step 1: Find the best prototype coalition (set of nodes forming the prototype)
    best_prototype_index, best_coalition, _ = find_best_prototype(
        data, gnnNet, prototype_vectors, prototype_vectors.size(0)
    )

    # Step 2: Convert the graph to NetworkX format
    graph = to_networkx(data, to_undirected=True)

    # Step 3: Create a subgraph using the best_coalition (node indices in the prototype)
    prototype_subgraph_nx = graph.subgraph(best_coalition)

    # Step 4: Retain the node features for the nodes in the subgraph
    # Create a mapping from NetworkX node indices to PyG node features
    node_features_subgraph = data.x[best_coalition]

    # Step 5: Add node features back to the NetworkX subgraph
    for i, node in enumerate(prototype_subgraph_nx.nodes()):
        prototype_subgraph_nx.nodes[node]["feat"] = node_features_subgraph[i]

    # Step 6: Convert the NetworkX subgraph back to a PyG Data object
    prototype_subgraph_pyg = from_networkx(prototype_subgraph_nx)

    # Step 7: Assign node features in PyG format
    prototype_subgraph_pyg.x = node_features_subgraph

    # Return the PyG subgraph with node features
    return prototype_subgraph_pyg


# graph_index = 309
# data = dataset[graph_index]


def run(data, idx):
    prototype = save_prototype_subgraph(data, gnnNets, prototype_vectors)
    torch.save(
        (data, prototype),
        f"protgnn_ba2motifs_graphs/graph_{idx}.pt",
    )


test_set = torch.load("ba2motifs_test_split.pt", weights_only=False)
class0 = [g for g in test_set if g.y == 0]

for i, g in enumerate(class0):
    run(g, 100 + i)